<a href="https://colab.research.google.com/github/AchrafAsh/gnn-receptive-fields/blob/main/02_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup environment

In [1]:
# setup colab environment
import os, sys
import os.path as osp
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/mnt


### Load Cora dataset

In [2]:
%%capture
!wget https://raw.githubusercontent.com/AchrafAsh/gnn-receptive-fields/main/data.py

In [3]:
from data import load_dataset

path = osp.join(os.getcwd(), 'data')
cora_dataset = load_dataset(path, 'Cora')
G = cora_dataset[0] # only graph of the dataset

Processing...
Done!


## Utils

In [4]:
import torch
import torch.nn.functional as F

from typing import Tuple, Dict, List
from tqdm import tqdm

In [22]:
def mean_average_distance(x, mask=None) -> float:
    D = 1 - torch.div(torch.matmul(x, torch.transpose(x, 0, 1)),
                      torch.matmul(torch.transpose(torch.norm(x, dim=1, keepdim=True), 0, 1), torch.norm(x, dim=1, keepdim=True)))
    if mask is not None:
        D = D * mask
    D_hat = torch.div( torch.sum(D, dim=1), torch.sum(D > 0, dim=1) )
    return torch.sum(D_hat) / torch.sum(D_hat > 0)

In [6]:
def count_parameters(model: torch.nn.Module):
    print(f"The model has {sum(p.numel() for p in model.parameters() if p.requires_grad):,} parameters")

In [7]:
def train(model: torch.nn.Module, optimizer: torch.optim.Optimizer,
          data: type(G), edge_index) -> Tuple[torch.tensor, float]:
    
    model.train()
    optimizer.zero_grad()

    hidden_state, out = model(data.x, edge_index)
    
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return hidden_state, loss

In [8]:
def evaluate(model: torch.nn.Module, data: type(G), edge_index) -> Dict[str, float]:
    
    model.eval()

    with torch.no_grad():
        _, logits = model(data.x, edge_index)
    
    outs = {}
    for key in ['train', 'val', 'test']:
        mask = data[f'{key}_mask']
        loss = F.nll_loss(logits[mask], data.y[mask]).item()
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()

        outs[f'{key}_loss'] = loss
        outs[f'{key}_acc'] = acc
    return outs

In [9]:
def run(data: type(G), model: torch.nn.Module,
        edge_index,
        runs: int, epochs: int, lr: float,
        weight_decay: float,
        early_stopping: int = 0,
        initialize:bool=True) -> Tuple[List[float], List[float], List[float]]:

    val_losses, accs, durations = [], [], []
    for _ in range(runs):
        data = data.to(device)
        model.to(device)
        if initialize:
            model.reset_parameters()
        
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, 
                                     weight_decay=weight_decay)

        if torch.cuda.is_available():
            torch.cuda.synchronize()
        
        t_start = time.perf_counter()

        best_val_loss = float('inf')
        val_loss_history, train_losses, test_accs, MADs = [], [], [], []

        for epoch in range(1, epochs+1):
            hidden_state, train_loss = train(model, optimizer, data, edge_index)

            MAD = mean_average_distance(x=hidden_state).item()
            eval_info = evaluate(model, data, edge_index)
            
            test_acc = eval_info['test_acc']
            val_loss_history.append(eval_info['val_loss'])
            train_losses.append(train_loss.item())
            test_accs.append(test_acc)
            MADs.append(MAD)
        
            # I don't understand what this is for...
            if eval_info['val_loss'] < best_val_loss:
                best_val_loss = eval_info['val_loss']
            
            if early_stopping > 0 and epoch > epochs // 2:
                tmp = torch.tensor(val_loss_history[-(early_stopping + 1):-1])
                if eval_info['val_loss'] > tmp.mean().item():
                    break
            
            if epoch % 10 == 0:
                print(f"Epoch: [{epoch} / {epochs}] | Loss: {train_loss} | Test accuracy: {test_acc} | MAD: {MAD}")

        if torch.cuda.is_available():
            torch.cuda.synchronize()

        t_end = time.perf_counter()

        val_losses.append(best_val_loss)
        accs.append(test_acc)
        durations.append(t_end - t_start)

    loss, acc, duration = torch.tensor(val_losses), torch.tensor(accs), torch.tensor(durations)

    print(f"Val Loss: {loss.mean().item():.4f}, Test Accuracy: {acc.mean().item():.3f} ± {acc.std().item():.3f}, Duration: {duration.mean().item():.3f}")
    return train_losses, test_accs, MADs

## Benchmark Models

### Simple GCN

In [26]:
import torch
from torch_geometric.nn import Sequential, GCNConv, GATConv

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
class GCN(torch.nn.Module):
    def __init__(self, num_layers:int, hidden_dim:int, num_features:int, num_classes:int, dropout:float=0.5):
        super().__init__()
        self.conv_layers = self.create_layers(num_layers=num_layers, 
                                              num_features=num_features,
                                              num_classes=num_classes,
                                              hidden_dim=hidden_dim,
                                              dropout=dropout)
        
        self.log_softmax = torch.nn.LogSoftmax(dim=1)


    def create_layers(self, num_layers:int, num_features:int, num_classes:int, 
                      hidden_dim:int, dropout:float):
        layers = []

        # first layer
        layers += [
                (GCNConv(in_channels=num_features, out_channels=hidden_dim), "x, edge_index -> x"),
                (torch.nn.ReLU(), "x -> x"),
                (torch.nn.Dropout(p=dropout), "x -> x")
        ]

        for k in range(1, num_layers-1):
            layers += [
                (GCNConv(in_channels=hidden_dim, out_channels=hidden_dim), "x, edge_index -> x"),
                (torch.nn.ReLU(), "x -> x"),
                (torch.nn.Dropout(p=dropout), "x -> x")
            ]
        
        # last layer
        layers += [
            (GCNConv(in_channels=hidden_dim, out_channels=num_classes), "x, edge_index -> x"),
        ]
        return Sequential("x, edge_index", layers)


    def reset_parameters(self):
        self.conv_layers.reset_parameters()


    def forward(self, x, edge_index):
        h = self.conv_layers(x, edge_index)
        return h, self.log_softmax(h)

### Simple GAT

In [33]:
class GAT(torch.nn.Module):
    def __init__(self, num_layers:int, hidden_dim:int, num_features:int, num_classes:int, dropout:float=0.5):
        super().__init__()
        self.conv_layers = self.create_layers(num_layers=num_layers, 
                                              num_features=num_features,
                                              num_classes=num_classes,
                                              hidden_dim=hidden_dim,
                                              dropout=dropout)
        
        self.log_softmax = torch.nn.LogSoftmax(dim=1)


    def create_layers(self, num_layers:int, num_features:int, num_classes:int, 
                      hidden_dim:int, dropout:float):
        layers = []

        # first layer
        layers += [
                (GATConv(in_channels=num_features, out_channels=hidden_dim, add_self_loops=True), "x, edge_index -> x"),
                (torch.nn.ReLU(), "x -> x"),
                (torch.nn.Dropout(p=dropout), "x -> x")
        ]

        for k in range(1, num_layers-1):
            layers += [
                (GATConv(in_channels=hidden_dim, out_channels=hidden_dim, add_self_loops=True), "x, edge_index -> x"),
                (torch.nn.ReLU(), "x -> x"),
                (torch.nn.Dropout(p=dropout), "x -> x")
            ]
        
        # last layer
        layers += [
            (GATConv(in_channels=hidden_dim, out_channels=num_classes, add_self_loops=True), "x, edge_index -> x"),
        ]
        return Sequential("x, edge_index", layers)


    def reset_parameters(self):
        self.conv_layers.reset_parameters()


    def forward(self, x, edge_index):
        h = self.conv_layers(x, edge_index)
        return h, self.log_softmax(h)

## Experiments

In [20]:
import time

### GCN

In [13]:
NUM_LAYERS=2
HIDDEN_DIM=16
NUM_FEATURES=cora_dataset.num_features
NUM_CLASSES=cora_dataset.num_classes
EPOCHS=200
LR=0.01
WEIGHT_DECAY=5e-4
EARLY_STOPPING=0
RUNS=1

In [16]:
model = GCN(num_layers=NUM_LAYERS, hidden_dim=HIDDEN_DIM, num_features=NUM_FEATURES, num_classes=NUM_CLASSES)
count_parameters(model)

The model has 23,063 parameters


In [24]:
train_loss, test_acc, mads = run(data=G, model=model,
    edge_index=G.edge_index,
    runs=RUNS, epochs=EPOCHS,
    lr=LR,
    weight_decay=WEIGHT_DECAY,
    early_stopping=EARLY_STOPPING)

Epoch: [10 / 200] | Loss: 0.7720257639884949 | Test accuracy: 0.791 | MAD: 0.9999647736549377
Epoch: [20 / 200] | Loss: 0.2279360294342041 | Test accuracy: 0.79 | MAD: 0.999981701374054
Epoch: [30 / 200] | Loss: 0.10261908173561096 | Test accuracy: 0.783 | MAD: 0.9999882578849792
Epoch: [40 / 200] | Loss: 0.046314895153045654 | Test accuracy: 0.78 | MAD: 0.999990701675415
Epoch: [50 / 200] | Loss: 0.037777215242385864 | Test accuracy: 0.779 | MAD: 0.9999918937683105
Epoch: [60 / 200] | Loss: 0.03894028812646866 | Test accuracy: 0.8 | MAD: 0.9999887943267822
Epoch: [70 / 200] | Loss: 0.050412822514772415 | Test accuracy: 0.781 | MAD: 0.999991238117218
Epoch: [80 / 200] | Loss: 0.046052269637584686 | Test accuracy: 0.793 | MAD: 0.9999871850013733
Epoch: [90 / 200] | Loss: 0.0427393838763237 | Test accuracy: 0.79 | MAD: 0.9999887943267822
Epoch: [100 / 200] | Loss: 0.045612137764692307 | Test accuracy: 0.807 | MAD: 0.9999887943267822
Epoch: [110 / 200] | Loss: 0.033620432019233704 | Test 

In [25]:
evaluate(model=model, data=G, edge_index=G.edge_index)

{'test_acc': 0.812,
 'test_loss': 0.6475297212600708,
 'train_acc': 1.0,
 'train_loss': 0.005584354512393475,
 'val_acc': 0.77,
 'val_loss': 0.7723254561424255}

### GAT

In [35]:
model = GAT(num_layers=NUM_LAYERS, hidden_dim=HIDDEN_DIM, num_features=NUM_FEATURES, num_classes=NUM_CLASSES)
count_parameters(model)

The model has 23,109 parameters


In [36]:
train_loss, test_acc, mads = run(data=G, model=model,
    edge_index=G.edge_index,
    runs=RUNS, epochs=EPOCHS,
    lr=LR,
    weight_decay=WEIGHT_DECAY,
    early_stopping=EARLY_STOPPING)

Epoch: [10 / 200] | Loss: 0.7547716498374939 | Test accuracy: 0.698 | MAD: 0.9999293088912964
Epoch: [20 / 200] | Loss: 0.25597402453422546 | Test accuracy: 0.739 | MAD: 0.9999663829803467
Epoch: [30 / 200] | Loss: 0.1451188027858734 | Test accuracy: 0.745 | MAD: 0.9999790191650391
Epoch: [40 / 200] | Loss: 0.042686790227890015 | Test accuracy: 0.737 | MAD: 0.9999762773513794
Epoch: [50 / 200] | Loss: 0.0530322901904583 | Test accuracy: 0.732 | MAD: 0.999971866607666
Epoch: [60 / 200] | Loss: 0.035886067897081375 | Test accuracy: 0.707 | MAD: 0.9999650120735168
Epoch: [70 / 200] | Loss: 0.020516367629170418 | Test accuracy: 0.728 | MAD: 0.9999730587005615
Epoch: [80 / 200] | Loss: 0.12050115317106247 | Test accuracy: 0.712 | MAD: 0.9999657273292542
Epoch: [90 / 200] | Loss: 0.027525370940566063 | Test accuracy: 0.705 | MAD: 0.9999648332595825
Epoch: [100 / 200] | Loss: 0.04233483597636223 | Test accuracy: 0.721 | MAD: 0.9999732971191406
Epoch: [110 / 200] | Loss: 0.02293691597878933 | 

In [37]:
evaluate(model=model, data=G, edge_index=G.edge_index)

{'test_acc': 0.749,
 'test_loss': 1.0232517719268799,
 'train_acc': 1.0,
 'train_loss': 0.002923202933743596,
 'val_acc': 0.708,
 'val_loss': 1.2085360288619995}